# Imports

In [23]:
import pandas as pd
from datetime import datetime, timedelta

In [24]:
import time
import os
import base64
import hashlib
import geopandas as gpd
import requests
import re
from dotenv import load_dotenv
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection, Point

In [25]:
import numpy as np                           

In [26]:
!pip install fancyimpute --user

# Data Cleaning

In [27]:
df_pas = pd.read_csv('Data/Property_Assessment/uniteevaluationfonciere.csv')

In [28]:
gdf_limadmin = gpd.read_file('Data/Boroughs/LIMADMIN.shp')


In [29]:
gdf_limadmin.describe(include='all')

,MUNID,CODEID,CODEMAMROT,NOM,TYPE,ABREV,NUM,AIRE,PERIM,geometry
count,34.0,34,34,34,34,34,34.000000,3.400000e+01,34.000000,34
unique,NaN,34,34,34,2,34,NaN,NaN,NaN,34
top,NaN,10,66097,Côte-Saint-Luc,Arrondissement,AJ,NaN,NaN,NaN,POLYGON ((-73.76363212758845 45.51193449332717...
freq,NaN,1,1,1,19,1,NaN,NaN,NaN,1
mean,66023.0,NaN,NaN,NaN,NaN,NaN,26.352941,1.817797e+07,22184.255500,NaN
std,0.0,NaN,NaN,NaN,NaN,NaN,25.630099,1.210695e+07,10636.682481,NaN
min,66023.0,NaN,NaN,NaN,NaN,NaN,1.000000,1.805089e+05,1814.424000,NaN
25%,66023.0,NaN,NaN,NaN,NaN,NaN,9.250000,8.535644e+06,16348.393250,NaN
50%,66023.0,NaN,NaN,NaN,NaN,NaN,17.500000,1.618194e+07,20747.373000,NaN
75%,66023.0,NaN,NaN,NaN,NaN,NaN,25.750000,2.512858e+07,28067.760250,NaN


In [30]:
gdf_limadmin.head(35)

,MUNID,CODEID,CODEMAMROT,NOM,TYPE,ABREV,NUM,AIRE,PERIM,geometry
0,66023,11,REM05,Outremont,Arrondissement,OM,5,3.813356e+06,10836.670,"POLYGON ((-73.62078 45.52365, -73.62020 45.524..."
1,66023,22,REM17,LaSalle,Arrondissement,LS,18,2.519727e+07,25259.849,"POLYGON ((-73.66610 45.42180, -73.66601 45.428..."
2,66023,62,66072,Mont-Royal,Ville liée,MR,2,7.445560e+06,18314.038,"POLYGON ((-73.65075 45.52631, -73.65036 45.526..."
3,66023,9,REM19,Ville-Marie,Arrondissement,VM,20,2.150063e+07,26585.959,"POLYGON ((-73.53013 45.53476, -73.53019 45.534..."
4,66023,5,REM21,Le Plateau-Mont-Royal,Arrondissement,PM,22,8.151665e+06,13158.328,"POLYGON ((-73.55923 45.53990, -73.56017 45.538..."
5,66023,54,66062,Hampstead,Ville liée,HS,10,1.768055e+06,5875.848,"POLYGON ((-73.65601 45.47893, -73.65601 45.481..."
6,66023,63,REM20,Le Sud-Ouest,Arrondissement,SO,21,1.814427e+07,29633.161,"POLYGON ((-73.62908 45.44839, -73.63197 45.449..."
7,66023,57,REM33,Rivière-des-Prairies-Pointe-aux-Trembles,Arrondissement,RP,19,5.004700e+07,38573.067,"POLYGON ((-73.62475 45.63359, -73.62469 45.633..."
8,66023,28,REM27,Lachine,Arrondissement,LC,17,2.312779e+07,25399.526,"POLYGON ((-73.72299 45.42179, -73.72085 45.438..."
9,66023,51,66087,Dorval,Ville liée,DV,1,2.815615e+07,32357.566,"POLYGON ((-73.79470 45.48181, -73.78960 45.484..."


In [31]:
# Dictionary where keys are 'CODEMAMROT' from the 'limadmin.shp' file and the values are the municipality numbers from the property assessment '.csv' file
dict_CODEMAMROT_municipality = {
    13: 66072,
    7: 66062,
    6: 66087,
    10: 66102,
    5: 66142,
    22: 66127,
    4: 66058,
    15: 66047,
    20: 66097,
    9: 66092,
    3: 66107,
    29: 66032,
    14: 66007,
    23: 66117,
    2: 66112
}

In [32]:
value = dict_CODEMAMROT_municipality.get(13, '')
value

66072

In [33]:
def map_values_to_new_column(df, gdf, dict_mapping, new_column_name):
    """
    Map values to a new column in a Pandas DataFrame based on the values
    in the 'NO_ARROND_ILE_CUM' and 'MUNICIPALITE' columns.
    
    Args:
        df (pandas.DataFrame): The Pandas DataFrame to map values for.
        gdf (geopandas.GeoDataFrame): The GeoDataFrame to use for mapping
            values based on the 'CODEMAMROT' column.
        dict_mapping (dict): The dictionary to use for mapping values
            based on the 'MUNICIPALITE' column.
        new_column_name (str): The name of the new column to create.
    """

    # create the new column in the Pandas DataFrame
    df[new_column_name] = ''

    # iterate over each row in the DataFrame
    for index, row in df.iterrows():

        # get the value of the 'NO_ARROND_ILE_CUM' column for this row
        no_arrond_ile_cum = row['NO_ARROND_ILE_CUM']

        # check if the value is 'REM99'
        if no_arrond_ile_cum == 'REM99':

            # get the value of the 'MUNICIPALITE' column for this row
            municipalite = row['MUNICIPALITE'] 

            # use the municipalite value as a key to search the dictionary
            value = dict_mapping.get(municipalite, '')

            # set the value of the new column for this row
            df.at[index, new_column_name] = value

        else:

            # use the value of the 'NO_ARROND_ILE_CUM' column directly
            df.at[index, new_column_name] = no_arrond_ile_cum

        # check if the 'CODEMAMROT' column is in the GeoDataFrame
        if 'CODEMAMROT' in gdf.columns:

            # get the value of the 'CODEMAMROT' column for this row
            codemamrot = row['CODEMAMROT']

            # use the codemamrot value as a key to search the GeoDataFrame
            value = gdf[gdf['CODEMAMROT'] ==
                        codemamrot][new_column_name].values

            # set the value of the new column for this row
            if len(value) > 0:
                df.at[index, new_column_name] = value[0]
    
    return df


In [34]:
df_pas = map_values_to_new_column(
    df_pas, gdf_limadmin, dict_CODEMAMROT_municipality, 'CODEMAMROT')


In [35]:
df_pas.columns

Index(['ID_UEV', 'CIVIQUE_DEBUT', 'CIVIQUE_FIN', 'NOM_RUE', 'SUITE_DEBUT',
       'MUNICIPALITE', 'ETAGE_HORS_SOL', 'NOMBRE_LOGEMENT',
       'ANNEE_CONSTRUCTION', 'CODE_UTILISATION', 'LETTRE_DEBUT', 'LETTRE_FIN',
       'LIBELLE_UTILISATION', 'CATEGORIE_UEF', 'MATRICULE83',
       'SUPERFICIE_TERRAIN', 'SUPERFICIE_BATIMENT', 'NO_ARROND_ILE_CUM',
       'CODEMAMROT'],
      dtype='object')

In [36]:
df_pas.CODEMAMROT.unique()

array(['REM20', 66032, 'REM19', 'REM33', 'REM12', 'REM22', 'REM25', 66097,
       'REM34', 'REM15', 66142, 'REM23', 'REM21', 'REM24', 'REM32',
       'REM31', 'REM27', 66072, 'REM05', 66107, 'REM16', 'REM09', 66102,
       'REM17', 66058, 'REM14', 66087, 66112, 66127, 66007, 66047, 66117,
       66062, 66092], dtype=object)

In [37]:
df_pas.describe(include='all')

,ID_UEV,CIVIQUE_DEBUT,CIVIQUE_FIN,NOM_RUE,SUITE_DEBUT,MUNICIPALITE,ETAGE_HORS_SOL,NOMBRE_LOGEMENT,ANNEE_CONSTRUCTION,CODE_UTILISATION,LETTRE_DEBUT,LETTRE_FIN,LIBELLE_UTILISATION,CATEGORIE_UEF,MATRICULE83,SUPERFICIE_TERRAIN,SUPERFICIE_BATIMENT,NO_ARROND_ILE_CUM,CODEMAMROT
count,5.015910e+05,501591.000000,501591.000000,501591,124441,501591.000000,449547.000000,454350.000000,501591.000000,501591.000000,3249,6302,501591,501591,501591,5.015900e+05,357830.000000,501591,501591
unique,NaN,NaN,NaN,6752,5312,NaN,NaN,NaN,NaN,NaN,15,17,585,3,501586,NaN,NaN,20,34
top,NaN,NaN,NaN,rue Sherbrooke Est (MTE+MTL),201,NaN,NaN,NaN,NaN,NaN,A,A,Logement,Régulier,9999-99-9999-9-999-9999,NaN,NaN,REM99,REM19
freq,NaN,NaN,NaN,2727,2813,NaN,NaN,NaN,NaN,NaN,2469,4998,452921,318535,6,NaN,NaN,76675,42241
mean,3.495348e+06,7306.062397,7307.347335,NaN,NaN,44.019121,1.688050,2.210296,2340.416551,1432.515789,NaN,NaN,NaN,NaN,NaN,8.071592e+02,826.035031,NaN,NaN
std,1.457486e+06,17231.523557,17231.329844,NaN,NaN,14.424152,0.816728,9.592524,1685.962906,1576.058864,NaN,NaN,NaN,NaN,NaN,1.756007e+04,9459.811433,NaN,NaN
min,1.000002e+06,1.000000,1.000000,NaN,NaN,2.000000,1.000000,1.000000,1600.000000,1000.000000,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN
25%,2.081410e+06,790.000000,790.000000,NaN,NaN,50.000000,1.000000,1.000000,1952.000000,1000.000000,NaN,NaN,NaN,NaN,NaN,7.100000e+01,139.000000,NaN,NaN
50%,4.008422e+06,3621.000000,3624.000000,NaN,NaN,50.000000,2.000000,1.000000,1972.000000,1000.000000,NaN,NaN,NaN,NaN,NaN,2.420000e+02,201.000000,NaN,NaN
75%,5.090688e+06,7140.000000,7140.000000,NaN,NaN,50.000000,2.000000,2.000000,2003.000000,1000.000000,NaN,NaN,NaN,NaN,NaN,4.490000e+02,288.000000,NaN,NaN


In [38]:
df_pas.shape

(501591, 19)

In [39]:
df_pas.dtypes

ID_UEV                   int64
CIVIQUE_DEBUT            int64
CIVIQUE_FIN              int64
NOM_RUE                 object
SUITE_DEBUT             object
MUNICIPALITE             int64
ETAGE_HORS_SOL         float64
NOMBRE_LOGEMENT        float64
ANNEE_CONSTRUCTION       int64
CODE_UTILISATION         int64
LETTRE_DEBUT            object
LETTRE_FIN              object
LIBELLE_UTILISATION     object
CATEGORIE_UEF           object
MATRICULE83             object
SUPERFICIE_TERRAIN     float64
SUPERFICIE_BATIMENT    float64
NO_ARROND_ILE_CUM       object
CODEMAMROT              object
dtype: object

In [40]:
df_pas = df_pas.drop(['CIVIQUE_FIN', 'CIVIQUE_DEBUT', 'NOM_RUE', 'SUITE_DEBUT', 'MUNICIPALITE', 'CODE_UTILISATION',
                     'LETTRE_DEBUT', 'LETTRE_FIN', 'LIBELLE_UTILISATION', 'MATRICULE83', 'NO_ARROND_ILE_CUM'], axis=1)

In [41]:
df_pas.dtypes

ID_UEV                   int64
ETAGE_HORS_SOL         float64
NOMBRE_LOGEMENT        float64
ANNEE_CONSTRUCTION       int64
CATEGORIE_UEF           object
SUPERFICIE_TERRAIN     float64
SUPERFICIE_BATIMENT    float64
CODEMAMROT              object
dtype: object

In [42]:
df_pas.isnull().sum()

ID_UEV                      0
ETAGE_HORS_SOL          52044
NOMBRE_LOGEMENT         47241
ANNEE_CONSTRUCTION          0
CATEGORIE_UEF               0
SUPERFICIE_TERRAIN          1
SUPERFICIE_BATIMENT    143761
CODEMAMROT                  0
dtype: int64

In [43]:
df_pas.shape

(501591, 8)

In [44]:
df_pas.to_csv('Data/Processed_Datasets/Cleaned_datasets/df_property_assessment.csv')